# Open Street Map Jogging route planner

In [45]:
import requests
import json
from dataclasses import dataclass
from typing import Dict, Tuple
import xml.etree.ElementTree as ET
import math
import random
import heapq

In [46]:
# 1. get Street and Node data from OSM
# 2. User inputs start and end points and route length
# 3.1 select a random Point
# 3.2-> use A* to find the path ot that random point
# 4. repeat 3. until route length is reached

In [47]:
#define dataclasses to store the data organized
@dataclass
class Node:
    id: int
    lat: float
    lon: float
    edges: list
    
    def __str__(self) -> str:
        return f"Node {self.id} ({self.lat}, {self.lon}), {len(self.edges)} edges"

@dataclass
class Edge:
    id: int
    start: Node
    end: Node
    length: float
    
    def __init__(self, id, start, end, length):
        self.id = id
        self.start = start
        self.end = end
        self.length = length
        
        start.edges.append(self)
        end.edges.append(self)
    
    def __str__(self) -> str:
        return f"Edge {self.id} ({self.start} -> {self.end})"

In [48]:
# get data from OSM
# store the data in the dataclasses
def get_data(pos1: Tuple[float, float], pos2: Tuple[float, float] ) -> Tuple[Dict[int, Node], Dict[int, Edge]]:
    """Get data from OSM and store it in the dataclasses"""
    URL = f"https://api.openstreetmap.org/api/0.6/map?bbox={pos1[1]},{pos1[0]},{pos2[1]},{pos2[0]}"
    raw_data = requests.get(URL).text
    
    nodes = {}  # id -> Node
    edges = {}  # id -> Edge
    
    # Parse nodes
    tree = ET.fromstring(raw_data)
    for node_elem in tree.iter("node"):
        node_id = int(node_elem.get("id"))
        node_lat = float(node_elem.get("lat"))
        node_lon = float(node_elem.get("lon"))
        node_edges = []
        nodes[node_id] = Node(id=node_id, lat=node_lat, lon=node_lon, edges=node_edges)
    
    # Parse edges
    for way_elem in tree.iter("way"):
        way_id = int(way_elem.get("id"))
        way_nodes = way_elem.findall("nd")
        way_length = 0.0
        start_node = None
        end_node = None
        
        for i in range(len(way_nodes) - 1):
            start_node_id = int(way_nodes[i].get("ref"))
            end_node_id = int(way_nodes[i+1].get("ref"))
            
            if start_node_id not in nodes or end_node_id not in nodes:
                continue
            
            start_node = nodes[start_node_id]
            end_node = nodes[end_node_id]
            
            distance = math.sqrt((start_node.lat - end_node.lat)**2 + (start_node.lon - end_node.lon)**2)
            way_length += distance
            
            edge = Edge(id=way_id, start=start_node, end=end_node, length=distance)
            edges[way_id] = edge
            
        if start_node and end_node:
            start_node.edges.append(edge)
            end_node.edges.append(edge)
    
    return nodes, edges

In [49]:
def haversine(lat_1, lon_1, lat_2, lon_2) -> float:
    #returns the distance between two points in km
    return math.sqrt((lat_1 - lat_2)**2 + (lon_1 - lon_2)**2)

In [50]:
nodes = {} #Dict[int, Node]
edges = {} #Dict[int, Edge]
nodes, edges = get_data((48.137154, 11.576124), (48.142893, 11.587868))

In [ ]:
print(random.choice(list(nodes.values())))
print(random.choice(list(nodes.values())))
print(random.choice(list(edges.values())))

In [52]:
def A_star(nodes: Dict[int, Node], edges: Dict[int, Edge], start_pos: Tuple[float, float], target_pos: Tuple[float, float]) -> Tuple[float, list]: #Distance, List of Node ids
    """A* algorithm to find the shortest path from the start node to the target node"""
    # Find start and target node
    target_node = sorted(nodes.values(), key=lambda node: math.sqrt((node.lat - target_pos[0])**2 + (node.lon - target_pos[1])**2))[0] #Find the closest node to the target position
    start_node = sorted(nodes.values(), key=lambda node: math.sqrt((node.lat - start_pos[0])**2 + (node.lon - start_pos[1])**2))[0] #Find the closest node to the start position

In [54]:
def experimental_A_star(nodes: Dict[int, Node], edges: Dict[int, Edge], start_pos: Tuple[float, float], target_pos: Tuple[float, float]) -> Tuple[float, list]:
    """A* algorithm to find the shortest path from the start node to the target node"""
    # Find start and target node
    target_node = sorted(nodes.values(), key=lambda node: math.sqrt((node.lat - target_pos[0])**2 + (node.lon - target_pos[1])**2))[0] # Find the closest node to the target position
    start_node = sorted(nodes.values(), key=lambda node: math.sqrt((node.lat - start_pos[0])**2 + (node.lon - start_pos[1])**2))[0] # Find the closest node to the start position

    # Initialize distances and previous nodes
    distance = {node.id: math.inf for node in nodes.values()}
    previous = {node.id: None for node in nodes.values()}
    distance[start_node.id] = 0

    # Priority queue for nodes to be visited
    queue = [(0, start_node)]

    while queue:
        _, current_node = heapq.heappop(queue)

        if current_node == target_node:
            # Reconstruct the path
            path = []
            while current_node is not None:
                path.append(current_node.id)
                current_node = previous[current_node.id]
            path.reverse()
            return distance[target_node.id], path

        for edge in current_node.edges:
            neighbor = edge.start if edge.end == current_node else edge.end
            neighbor_distance = distance[current_node.id] + edge.length
            if neighbor_distance < distance[neighbor.id]:
                distance[neighbor.id] = neighbor_distance
                previous[neighbor.id] = current_node
                heapq.heappush(queue, (neighbor_distance + haversine(neighbor.lat, neighbor.lon, target_node.lat, target_node.lon), neighbor))

    # No path found
    return math.inf, []

In [55]:
distance, path = experimental_A_star(nodes, edges, (48.1386717, 11.5782439), (48.1401162, 11.5873112))

In [ ]:
path_edges = [edges[edge_id] for edge_id in path]